# HFSS-Mechanical multiphysics analysis

This example shows how to use PyAEDT to create a multiphysics workflow that
includes Circuit, HFSS, and Mechanical.

Keywords: **Multiphysics**, **HFSS**, **Mechanical AEDT**, **Circuit**.

## Perform imports and define constants

Perform required imports.

In [1]:
import tempfile
import time

import ansys.aedt.core
from ansys.aedt.core.examples.downloads import download_via_wizard

Define constants.

In [2]:
AEDT_VERSION = "2025.2"
NUM_CORES = 4
NG_MODE = False  # Open AEDT UI when it is launched.

## Create temporary directory

Create a temporary directory where downloaded data or
dumped data can be stored.
If you'd like to retrieve the project data for subsequent use,
the temporary folder name is given by ``temp_folder.name``.

In [3]:
temp_folder = tempfile.TemporaryDirectory(suffix=".ansys")

## Download and open project

Download and open the project. Save it to the temporary folder.

In [4]:
project_name = download_via_wizard(
    local_path=temp_folder.name
)

## Start HFSS

Initialize HFSS.

In [5]:
hfss = ansys.aedt.core.Hfss(
    project=project_name,
    version=AEDT_VERSION,
    non_graphical=NG_MODE,
    new_desktop=True,
)
hfss.change_material_override(True)

PyAEDT INFO: Parsing C:\Users\ansys\AppData\Local\Temp\tmp4_rx6sk9.ansys\viawizard_vacuum_FR4.aedt.


PyAEDT INFO: Python version 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)].


PyAEDT INFO: PyAEDT version 0.25.dev0.


PyAEDT INFO: Initializing new Desktop session.


PyAEDT INFO: Log on console is enabled.


...

PyAEDT INFO: Active Design set to HFSSDesign1


PyAEDT INFO: Active Design set to HFSSDesign1


PyAEDT INFO: Aedt Objects correctly read


PyAEDT INFO: Enabling Material Override


True

## Initialize Circuit

Initialize Circuit and add the HFSS dynamic link component.

In [6]:
circuit = ansys.aedt.core.Circuit(version=AEDT_VERSION)
hfss_comp = circuit.modeler.schematic.add_subcircuit_dynamic_link(pyaedt_app=hfss)

PyAEDT INFO: Python version 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)].


PyAEDT INFO: PyAEDT version 0.25.dev0.


PyAEDT INFO: Returning found Desktop session with PID 8280!


PyAEDT INFO: No project is defined. Project viawizard_vacuum_FR4 exists and has been read.


PyAEDT INFO: No consistent unique design is present. Inserting a new design.


PyAEDT INFO: Added design 'Circuit Design_QSC' of type Circuit Design.


PyAEDT INFO: Aedt Objects correctly read


PyAEDT INFO: ModelerCircuit class has been initialized! Elapsed time: 0m 0sec


PyAEDT INFO: ModelerNexxim class has been initialized!


PyAEDT INFO: Modeler class has been initialized! Elapsed time: 0m 0sec


## Set up dynamic link options

Set up dynamic link options. The argument for the ``set_sim_option_on_hfss_subcircuit()``
method can be the component name, component ID, or component object.

In [7]:
circuit.modeler.schematic.refresh_dynamic_link(name=hfss_comp.composed_name)
circuit.modeler.schematic.set_sim_option_on_hfss_subcircuit(component=hfss_comp)
hfss_setup_name = hfss.setups[0].name + " : " + hfss.setups[0].sweeps[0].name
circuit.modeler.schematic.set_sim_solution_on_hfss_subcircuit(
    component=hfss_comp.composed_name, solution_name=hfss_setup_name
)

PyAEDT WARNING: Parameter 'ID' is invalid and will be ignored.


PyAEDT WARNING: Parameter 'MinSolvedFreq' is invalid and will be ignored.


True

## Create ports and excitations

Create ports and excitations. Find component pin locations and create interface
ports on them. Define the voltage source on the input port.

In [8]:
circuit.modeler.schematic.create_interface_port(
    name="Excitation_1",
    location=[hfss_comp.pins[0].location[0], hfss_comp.pins[0].location[1]],
)
circuit.modeler.schematic.create_interface_port(
    name="Excitation_2",
    location=[hfss_comp.pins[1].location[0], hfss_comp.pins[1].location[1]],
)
circuit.modeler.schematic.create_interface_port(
    name="Port_1",
    location=[hfss_comp.pins[2].location[0], hfss_comp.pins[2].location[1]],
)
circuit.modeler.schematic.create_interface_port(
    name="Port_2",
    location=[hfss_comp.pins[3].location[0], hfss_comp.pins[3].location[1]],
)

ports_list = ["Excitation_1", "Excitation_2"]
source = circuit.assign_voltage_sinusoidal_excitation_to_ports(ports_list)
source.ac_magnitude = 1
source.phase = 0

## Create setup

In [9]:
setup_name = "MySetup"
LNA_setup = circuit.create_setup(name=setup_name)
sweep_list = ["LINC", str(4.3) + "GHz", str(4.4) + "GHz", str(1001)]
LNA_setup.props["SweepDefinition"]["Data"] = " ".join(sweep_list)

## Solve and push excitations

Solve the circuit and push excitations to the HFSS model to calculate the
correct value of losses.

In [10]:
circuit.analyze(cores=NUM_CORES)
circuit.push_excitations(instance="S1", setup=setup_name)

PyAEDT INFO: Project viawizard_vacuum_FR4 Saved correctly


PyAEDT INFO: Key Desktop/ActiveDSOConfigurations/Circuit Design correctly changed.


PyAEDT INFO: Solving all design setups. Analysis started...


PyAEDT INFO: Design setup None solved correctly in 0.0h 2.0m 5.0s


PyAEDT INFO: Key Desktop/ActiveDSOConfigurations/Circuit Design correctly changed.


True

## Start Mechanical

Start Mechanical and copy bodies from the HFSS project.

In [11]:
mech = ansys.aedt.core.Mechanical(version=AEDT_VERSION)
mech.copy_solid_bodies_from(design=hfss)
mech.change_material_override(True)

PyAEDT INFO: Python version 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)].


PyAEDT INFO: PyAEDT version 0.25.dev0.


PyAEDT INFO: Returning found Desktop session with PID 8280!


PyAEDT INFO: No project is defined. Project viawizard_vacuum_FR4 exists and has been read.


PyAEDT INFO: No consistent unique design is present. Inserting a new design.


...

PyAEDT INFO: Refreshing bodies from Object Info


PyAEDT INFO: Bodies Info Refreshed Elapsed time: 0m 0sec


PyAEDT INFO: 3D Modeler objects parsed. Elapsed time: 0m 0sec


PyAEDT INFO: Enabling Material Override


True

## Get losses from HFSS and assign convection to Mechanical

In [12]:
mech.assign_em_losses(
    design=hfss.design_name,
    setup=hfss.setups[0].name,
    sweep="LastAdaptive",
    map_frequency=hfss.setups[0].props["Frequency"],
    surface_objects=hfss.get_all_conductors_names(),
)
diels = ["1_pd", "2_pd", "3_pd", "4_pd", "5_pd"]
for el in diels:
    mech.assign_uniform_convection(
        assignment=[mech.modeler[el].top_face_y, mech.modeler[el].bottom_face_y],
        convection_value=3,
    )

PyAEDT INFO: Parsing C:\Users\ansys\AppData\Local\Temp\tmp4_rx6sk9.ansys\viawizard_vacuum_FR4.aedt.


PyAEDT INFO: File C:\Users\ansys\AppData\Local\Temp\tmp4_rx6sk9.ansys\viawizard_vacuum_FR4.aedt correctly loaded. Elapsed time: 0m 0sec


PyAEDT INFO: aedt file load time 0.14286208152770996


PyAEDT INFO: Materials class has been initialized! Elapsed time: 0m 0sec


PyAEDT INFO: Mapping HFSS EM Loss


...

PyAEDT INFO: Boundary Convection Convection_HXCT7K has been created.


PyAEDT INFO: Boundary Convection Convection_H2N8XA has been created.


PyAEDT INFO: Boundary Convection Convection_TG3S8A has been created.


PyAEDT INFO: Boundary Convection Convection_ZTBU5V has been created.


PyAEDT INFO: Boundary Convection Convection_N8PREU has been created.


## Solve and plot thermal results

In [13]:
mech.create_setup()
mech.save_project()
mech.analyze(cores=NUM_CORES)
surfaces = []
for name in mech.get_all_conductors_names():
    surfaces.extend(mech.modeler.get_object_faces(name))
mech.post.create_fieldplot_surface(assignment=surfaces, quantity="Temperature")

PyAEDT INFO: Project viawizard_vacuum_FR4 Saved correctly


PyAEDT INFO: Project viawizard_vacuum_FR4 Saved correctly


PyAEDT INFO: Key Desktop/ActiveDSOConfigurations/Mechanical correctly changed.


PyAEDT INFO: Solving all design setups. Analysis started...


PyAEDT INFO: Design setup None solved correctly in 0.0h 1.0m 29.0s


...

PyAEDT INFO: aedt file load time 0.2470533847808838


PyAEDT INFO: PostProcessor class has been initialized! Elapsed time: 0m 0sec


PyAEDT INFO: Post class has been initialized! Elapsed time: 0m 0sec


PyAEDT INFO: Active Design set to Mechanical_QS4


Class: ansys.aedt.core.visualization.post.field_data.FieldPlot

## Release AEDT

Release AEDT and close the example.

In [14]:
mech.save_project()
mech.release_desktop()
# Wait 3 seconds to allow AEDT to shut down before cleaning the temporary directory.
time.sleep(3)

PyAEDT INFO: Project viawizard_vacuum_FR4 Saved correctly


PyAEDT INFO: Desktop has been released and closed.


## Clean up

All project files are saved in the folder ``temp_folder.name``. If you've run this example as a Jupyter notebook, you
can retrieve those project files. The following cell removes all temporary files, including the project folder.

In [15]:
temp_folder.cleanup()